<a href="https://colab.research.google.com/github/AaryanBairagi/GeekForGeeks-21Days-21Projects-Challenge/blob/main/Day_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install langchain
! pip install langchain-community
! pip install langchain-google-genai

# **TESTING- Simple CHATBOT**

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage , SystemMessage
from google.colab import userdata

chat_model = ChatGoogleGenerativeAI(model='gemini-2.5-flash',
                                    temperature=0,
                                    google_api_key=userdata.get('smart_IDP'))

messages = [
    SystemMessage("You are an assistant knowledgable about Healthcare. Only answer Healthcare-related questions."),
    HumanMessage(content="What is Pulmonary Artery?")
    ]

result = chat_model.invoke(messages)
print(result.content)


The **Pulmonary Artery** is a major blood vessel that originates from the right ventricle of the heart and carries deoxygenated blood to the lungs.

Here's a breakdown of its key characteristics and function:

1.  **Origin:** It emerges from the right ventricle of the heart.
2.  **Blood Type:** Unlike most other arteries in the body (which carry oxygenated blood), the pulmonary artery carries **deoxygenated blood** (blood that has already delivered its oxygen to the body's tissues and is rich in carbon dioxide).
3.  **Destination:** It branches into the right and left pulmonary arteries, which then further divide into smaller arteries and capillaries within the lungs.
4.  **Function:** Its primary role is to transport this deoxygenated blood to the lungs, where it can release carbon dioxide and pick up fresh oxygen through the process of gas exchange in the alveoli.
5.  **Unique Feature:** It is the only artery in the adult body that carries deoxygenated blood. Once the blood is oxygen

# **TESTING-Using Chains and PromptTemplates**

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from google.colab import userdata
from langchain_core.output_parsers import StrOutputParser

chat_model = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0,
    google_api_key=userdata.get('smart_IDP')
)

instruction_str = """Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed as possible, but don't make up any information that's not from the context.
If you don't know an answer, say you don't know.

Context: {context}

Question: {question}
"""

chat_template = ChatPromptTemplate.from_template(instruction_str)

context = "Discharge process was seamless!"
question = "Did anyone have a positive experience?"

chain = chat_template | chat_model | StrOutputParser()

result = chain.invoke({
    "context":context,
    "question":question
})

print(result)

Yes, someone had a positive experience. Their discharge process was seamless, indicating a smooth and problem-free experience.


# **Using PromptTemplates & MessageTemplates**

In [4]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (ChatPromptTemplate , SystemMessagePromptTemplate , HumanMessagePromptTemplate , PromptTemplate)

chat_model = ChatGoogleGenerativeAI(model='gemini-2.5-flash',
                                    temperature=0,
                                    google_api_key=userdata.get('smart_IDP'))

instruction_str = """Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions.
Be as detailed as possible, but don't make up any information that's not from the context.
If you don't know an answer, say you don't know.

Context: {context}
"""

chat_system_prompt_template = SystemMessagePromptTemplate(
    prompt = PromptTemplate(input_variables=["context"] , template=instruction_str)
)


chat_human_prompt_template = HumanMessagePromptTemplate(
    prompt = PromptTemplate(input_variables=["question"] , template="{question}")
)

messages=[chat_system_prompt_template,chat_human_prompt_template]

chat_prompt_template = ChatPromptTemplate(
    input_variables=["context","question"],
    messages=messages
)

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

chain = chat_prompt_template | chat_model | StrOutputParser()

result = chain.invoke({
    "context":context,
    "question":question
})

print(result)

Yes, one patient stated, "I had a great stay!"


# **Adding RAG**

In [5]:
! pip install langchain-chroma

In [6]:
# import time
# from google.colab import userdata
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_chroma import Chroma
# from langchain.document_loaders import CSVLoader

# review_csv_path = "/content/reviews.csv"

# review_chroma_path = "chroma_data"

# loader = CSVLoader(
#     file_path = review_csv_path,
#     source_column = 'review'
# )

# reviews = loader.load()

# embedding_function = GoogleGenerativeAIEmbeddings(
#     model='models/gemini-embedding-001',
#     google_api_key = userdata.get("smart_IDP")
# )

# batch_size = 20

# num_batches = (len(reviews)-1) // batch_size+1
# reviews_verctor_db = None

# for i in range(0,len(reviews),batch_size):
#   batch_docs = reviews[i:i+batch_size]
#   current_batch = i // batch_size + 1

#   print(f"Processing batch {current_batch}/{num_batches}...")
#   if i==0:
#     reviews_verctor_db = Chroma.from_documents(
#         documents=batch_docs,
#         embedding=embedding_function,
#         persist_directory=review_chroma_path
#     )

#   else:
#     reviews_verctor_db.add_documents(documents=batch_docs)

#   print(f"Batch {current_batch} processed. Waiting for 30 seconds...")
#   time.sleep(30)

# print("Vector database created successfully and saved to the specified directory.")


# **PUTTING IT ALL TOGETHER**

In [9]:
from langchain_core.prompts import( SystemMessagePromptTemplate , HumanMessagePromptTemplate , ChatPromptTemplate , PromptTemplate)
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain.document_loaders import CSVLoader
import time

instruction_str = """Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions.
Be as detailed as possible, but don't make up any information that's not from the context.
If you don't know an answer, say you don't know.

{context}
"""

reviews_system_prompt_template = SystemMessagePromptTemplate(
    prompt = PromptTemplate(input_variables=["context"],
    template = instruction_str)
)


reviews_human_prompt_template = HumanMessagePromptTemplate(
    prompt = PromptTemplate(input_variables=["question"],
    template = "{question}")
)


messages = [reviews_system_prompt_template , reviews_human_prompt_template]


reviews_chat_prompt_template = ChatPromptTemplate(
    input_variables=["context","question"],
    messages = messages
)


print(f"Creating Chat Model...")

reviews_chat_model = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash',
    temperature = 0,
    google_api_key = userdata.get('RAGBOT_API_KEY')
)


print(f"Chat Model Created.")


reviews_cs_path = "/content/reviews.csv"


loader = CSVLoader(
    file_path = reviews_cs_path,
    source_column = 'review'
)


reviews = loader.load()


reviews_chroma_path = "chroma_data"


embedding_function = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=userdata.get("RAGBOT_API_KEY")
)


batch_size = 20


num_of_batches = (len(reviews) - 1 ) // batch_size + 1


for i in range(0,len(reviews),batch_size):
  batch_docs = reviews[i:i+batch_size]
  current_batch = i // batch_size+1

  print(f"Processing batch {current_batch}/{num_of_batches}...")

  if i==0:
    reviews_vector_db = Chroma.from_documents(
        documents = batch_docs,
        embedding = embedding_function,
        persist_directory = reviews_chroma_path
    )
  else:
    reviews_vector_db.add_documents(documents=batch_docs)

  print(f"Batch {current_batch} processed. Waiting for 30 seconds...")
  time.sleep(30)

print("Vector database created successfully and saved to the specified directory")



Creating Chat Model...
Chat Model Created.
Processing batch 1/51...
Batch 1 processed. Waiting for 30 seconds...
Processing batch 2/51...
Batch 2 processed. Waiting for 30 seconds...
Processing batch 3/51...
Batch 3 processed. Waiting for 30 seconds...
Processing batch 4/51...
Batch 4 processed. Waiting for 30 seconds...
Processing batch 5/51...
Batch 5 processed. Waiting for 30 seconds...
Processing batch 6/51...
Batch 6 processed. Waiting for 30 seconds...
Processing batch 7/51...
Batch 7 processed. Waiting for 30 seconds...
Processing batch 8/51...
Batch 8 processed. Waiting for 30 seconds...
Processing batch 9/51...
Batch 9 processed. Waiting for 30 seconds...
Processing batch 10/51...
Batch 10 processed. Waiting for 30 seconds...
Processing batch 11/51...
Batch 11 processed. Waiting for 30 seconds...
Processing batch 12/51...
Batch 12 processed. Waiting for 30 seconds...
Processing batch 13/51...
Batch 13 processed. Waiting for 30 seconds...
Processing batch 14/51...
Batch 14 proc

In [10]:
from langchain.schema.runnable import RunnablePassthrough  # Allows passing inputs through unchanged in a pipeline
from langchain_core.output_parsers import StrOutputParser

reviews_retriever = reviews_vector_db.as_retriever(k=10)

review_chain = (
    {"context":reviews_retriever , "question":RunnablePassthrough()} | reviews_chat_prompt_template | reviews_chat_model | StrOutputParser()
)

In [11]:
question = """Has anyone complained about communication with the hospital staff?"""
review_chain.invoke(question)

GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerUserPerProjectPerModel-FreeTier"
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
]

In [12]:
! pip install gradio

In [13]:
def respond_to_user_question(question:str , history:list) -> str:
  """
  Respond the user's question using the review_chain
  """
  return review_chain.invoke(question)

In [14]:
respond_to_user_question("Has anyone complaied about communication with the hospital staff?",[])

GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerUserPerProjectPerModel-FreeTier"
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
]

In [ ]:
import gradio as gr
interface = gr.ChatInterface(fn=respond_to_user_question, title="Review Helper Bot")

interface.launch(debug=True)